In [ ]:
%pip install pytest
dbutils.library.restartPython()

In [ ]:
import os

os.chdir("/Workspace/Users/eric.liou@databricks.com/.bundle/ericliou_project/dev/files/src/tests/unit_test")
print("CWD:", os.getcwd())


In [ ]:
import importlib
print(importlib.import_module("test_myfunctions"))
print(importlib.import_module("myfunctions"))

In [ ]:
import pytest
import sys
import os
from datetime import datetime

sys.dont_write_bytecode = True

report_path = "./unit_test_report.xml"

os.makedirs(os.path.dirname(report_path), exist_ok=True)

retcode = pytest.main([
    ".", 
    "-v", 
    "-p", "no:cacheprovider",
    f"--junitxml={report_path}"
])

assert retcode == 0, "The pytest invocation failed. See the log for details."

In [ ]:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from xml.etree import ElementTree as ET
from datetime import datetime

schema = StructType([
    StructField("run_id", StringType(), True),
    StructField("test_class", StringType(), True),
    StructField("test_name", StringType(), True),
    StructField("duration_sec", DoubleType(), True),
    StructField("status", StringType(), True),
    StructField("message", StringType(), True),
])

report_path = "./unit_test_report.xml"
tree = ET.parse(report_path)
root = tree.getroot()

run_id = datetime.utcnow().isoformat()

rows = []
for testcase in root.iter("testcase"):
    classname = testcase.attrib.get("classname", "")
    name = testcase.attrib.get("name", "")
    time = float(testcase.attrib.get("time", 0.0))

    status = "passed"
    message = None

    failure = testcase.find("failure")
    error = testcase.find("error")
    skipped = testcase.find("skipped")

    if failure is not None:
        status = "failed"
        msg = failure.attrib.get("message", failure.text)
        message = str(msg) if msg is not None else None
    elif error is not None:
        status = "error"
        msg = error.attrib.get("message", error.text)
        message = str(msg) if msg is not None else None
    elif skipped is not None:
        status = "skipped"
        msg = skipped.attrib.get("message", skipped.text)
        message = str(msg) if msg is not None else None

    rows.append((
        run_id,
        str(classname),
        str(name),
        float(time),
        str(status),
        message
    ))

df = spark.createDataFrame(rows, schema=schema)

(
    df.write
      .format("delta")
      .mode("overwrite")
      .saveAsTable("eric_liou.default.unit_test_results")
)
